In [48]:
!pip install rapidfuzz


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [55]:
import os, sys, pandas as pd
from pyspark.sql import SparkSession

def log(msg): print(msg, file=sys.stdout, flush=True)

In [50]:
print('helloworltrdf')

helloworltrdf


In [56]:
# ── 1. Spark session ────────────────────────────────────────────────
spark = SparkSession.getActiveSession()
if spark is None:
    spark = (
        SparkSession.builder
        .appName("ReloadBronzeAug2021")
        .config("spark.driver.memory", "8g")
        .getOrCreate()
    )
spark.conf.set("spark.sql.debug.maxToStringFields", 200)

In [57]:

# ── 2. Locate bronze files (try "2021-8" then "2021-08") ────────────
bronze_dir = os.path.join(os.getcwd(), "datamart", "bronze")
snapshot   = "2021-8"                     # default pattern

paths = {
    "resume": os.path.join(bronze_dir, f"resume_{snapshot}.parquet"),
    "jd":     os.path.join(bronze_dir, f"jd_{snapshot}.parquet"),
    "labels": os.path.join(bronze_dir, f"labels_{snapshot}.parquet"),
}

# fallback to zero-padded filenames
if not all(os.path.exists(p) for p in paths.values()):
    snapshot = "2021-08"
    paths = {k: os.path.join(bronze_dir, f"{k}_{snapshot}.parquet")
             for k in ["resume", "jd", "labels"]}

# ensure every file exists
for name, p in paths.items():
    assert os.path.exists(p), f"{name.capitalize()} file missing: {p}"


In [58]:
# ── 3. Load Parquet shards ──────────────────────────────────────────
resume_df = spark.read.parquet(paths["resume"])
jd_df     = spark.read.parquet(paths["jd"])
labels_df = spark.read.parquet(paths["labels"])

log(f"Loaded RESUME  → rows: {resume_df.count():,}")
log(f"Loaded JD      → rows: {jd_df.count():,}")
log(f"Loaded LABELS  → rows: {labels_df.count():,}")

Loaded RESUME  → rows: 1,038
Loaded JD      → rows: 1,038
Loaded LABELS  → rows: 1,038


In [59]:
# ── 4. Show schemas & small samples ─────────────────────────────────
pd.set_option("display.max_colwidth", 250)

log("\n========== RESUME SCHEMA ==========")
resume_df.printSchema()

log("\n--- Nested `education` schema (resume) ---")
resume_df.select("education").printSchema()

log("\n--- Sample resume `education` arrays ---")
display(
    resume_df.select("id", "education")
             .limit(5)
             .toPandas()
)

log("\n========== JD SCHEMA ==========")
jd_df.printSchema()

log("\n--- Sample JD `required_education` strings ---")
display(
    jd_df.select("id", "required_education")
         .limit(5)
         .toPandas()
)

log("\n========== LABELS SCHEMA ==========")
labels_df.printSchema()

log("\n--- Sample label rows ---")
display(
    labels_df.limit(5).toPandas()
)


========== RESUME SCHEMA ==========
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location_preference: string (nullable = true)
 |-- work_authorization: string (nullable = true)
 |-- employment_type_preference: string (nullable = true)
 |-- hard_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- experience: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- role: string (nullable = true)
 |    |    |-- company: string (nullable = true)
 |    |    |-- date_start: string (nullable = true)
 |    |    |-- date_end: string (nullable = true)
 |    |    |-- role_description: string (nullable = true)
 |    |    |-- snapshot_date: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |-- education: ar

,id,education
0,RES_CHUA6p90,"[(Bachelor of Science, California State University Northridge, None, None, 3.6500000953674316, Healthcare Administration. Additional Coursework in Healthcare Leadership, Economy, Geriatric Care, Public Health, Biostatistics, Legal Health, IT and ..."
1,RES_SlLMkiqd,"[(Doctor of Public Health, George Washington University, Milken Institute School of Public Health, None, present, None, None, None, None), (Master of Science: Public Health, Tulane University School of Public Health, None, 2012-05-01T00:00:00, No..."
2,RES_mFb6f8iE,[]
3,RES_zkuNDo8d,"[(Bachelor of Technology, Mahatma Gandhi University, None, 2006-01-01T00:00:00, None, Computer Science and Engineering, None, None), (B.Tech - CSE, Mahatma Gandhi University, None, 2006-01-01T00:00:00, None, None, None, None), (None, Mahatma Gand..."
4,RES_PqbOkeex,[]



========== JD SCHEMA ==========
root
 |-- id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- role_title: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- about_the_company: string (nullable = true)
 |-- job_responsibilities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_hard_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_language_proficiencies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_education: string (nullable = true)
 |-- required_work_authorization: string (nullable = true)
 |-- certifications: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- snapshot_date: date (nullable = true)


--- Sample JD `required_education` strings ---


,id,required_education
0,JD_CHUA6p90,None
1,JD_SlLMkiqd,Bachelor's degree in computer science or a related discipline required. Master's degree preferred.
2,JD_mFb6f8iE,"BS or BA with emphasis in computer science, information systems, or business administration"
3,JD_zkuNDo8d,"A bachelor's degree in IT, Computer Science or related field or an MBA."
4,JD_PqbOkeex,Bachelor's Degree and six to ten years of experience or equivalent education and software engineering training or experience



========== LABELS SCHEMA ==========
root
 |-- _id: string (nullable = true)
 |-- resume_id: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- fit: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- fit_score: double (nullable = true)


--- Sample label rows ---


,_id,resume_id,job_id,fit,snapshot_date,fit_score
0,684547f570634603a66d8df5,RES_CHUA6p90,JD_CHUA6p90,No Fit,2021-08-11,0.0
1,684547f870634603a66d8df7,RES_SlLMkiqd,JD_SlLMkiqd,No Fit,2021-08-26,0.0
2,6845480470634603a66d8dfe,RES_mFb6f8iE,JD_mFb6f8iE,No Fit,2021-08-09,0.0
3,6845481670634603a66d8e09,RES_zkuNDo8d,JD_zkuNDo8d,No Fit,2021-08-09,0.0
4,6845481a70634603a66d8e0c,RES_PqbOkeex,JD_PqbOkeex,No Fit,2021-08-21,0.0


# Start imputation


In [1]:
# Cell 1 – (re-)create utils/education_parser.py  ───────────────────────
import os, textwrap
module_dir = os.path.join(os.getcwd(), "utils")
os.makedirs(module_dir, exist_ok=True)
parser_path = os.path.join(module_dir, "education_parser.py")

parser_code = """
import re, os, json, functools, requests
from rapidfuzz import fuzz, process
from sentence_transformers import SentenceTransformer, util
from dotenv import load_dotenv

# ─── degree-level rules ──────────────────────────────────────────────
LEVEL_SYNS = {
    4: ["phd","doctorate","doctoral","dphil","edd","doctor of"],
    3: ["master","msc","m.sc","m.s","mba","m.eng","m.tech"],
    2: ["bachelor","bsc","b.sc","bs ","ba ","b.tech","b.eng","be "],
    1: ["associate","aas","a.a.s","aa ","a.s.","associates"],
    0: ["high school","secondary","ged","h.s"]
}
LEVEL_CANON = {k:v[0].split()[0].title() for k,v in LEVEL_SYNS.items()}
LEVEL_RX    = {k:re.compile(rf"\\b({'|'.join(v)})\\b",re.I)
               for k,v in LEVEL_SYNS.items()}

def _rule_level(txt):
    for k in sorted(LEVEL_RX,reverse=True):
        if LEVEL_RX[k].search(txt): return k
    return None

FUZZ_TGT = ["Doctor","Master","Bachelor","Associate","High School"]
def _fuzzy_level(txt):
    best,score,_ = process.extractOne(txt,FUZZ_TGT,scorer=fuzz.WRatio)
    return FUZZ_TGT.index(best) if score>=88 else None

# ─── MiniLM embedding -- catches odd phrasings ───────────────────────
_embed      = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
_canon_emb  = _embed.encode([f"{LEVEL_CANON[k]} degree" for k in range(5)],
                            normalize_embeddings=True)
def _embed_level(txt):
    v = _embed.encode([txt],normalize_embeddings=True)
    sims = util.cos_sim(v,_canon_emb)[0]
    best = int(sims.argmax())
    return best if sims[best]>0.5 else None

# ─── canonical majors -- small synonym map (extend as needed) ────────
MAJOR_MAP = {
    "Computer Science": ["computer science","comp sci","cse","cs","software eng"],
    "Information Tech" : ["information technology","info tech"," it "],
    "Business Admin"   : ["business administration","business studies","bba"],
    "Electrical Eng"   : ["electrical eng","ee","electrical engineering"],
    "Mechanical Eng"   : ["mechanical eng"],
    "Accounting"       : ["accounting","accountancy"],
    "Data Science"     : ["data science"],
    "Public Health"    : ["public health","mph"]
}
def _canon_major(raw):
    if not raw: return None
    low=raw.lower()
    for canon,toks in MAJOR_MAP.items():
        if any(t in low for t in toks): return canon
    return None

# ─── Mistral fallback (only for degree level) ────────────────────────
load_dotenv()
_KEY=os.getenv("MISTRAL_API_KEY")
_URL="https://api.mistral.ai/v1/chat/completions"
_FUNC={"name":"parse_degree","parameters":{"type":"object","properties":{
       "level":{"type":"string"},"major":{"type":"string"},
       "is_degree":{"type":"boolean"}}}}

@functools.lru_cache(maxsize=5000)
def _mistral(prompt:str):
    if not (_KEY and prompt): return None
    body=dict(model="mistral-large-latest",temperature=0,
              messages=[{"role":"system","content":"Extract highest academic degree."},
                        {"role":"user","content":prompt}],
              tools=[{"type":"function","function":_FUNC}],
              tool_choice={"name":"parse_degree"})
    r=requests.post(_URL,json=body,headers={"Authorization":f"Bearer {_KEY}"},timeout=20)
    if r.status_code!=200: return None
    args=json.loads(r.json()["choices"][0]["message"]["tool_calls"][0]
                    ["function_call"]["arguments"])
    lvl_txt=(args.get("level") or "").lower()
    lvl=next((k for k,v in LEVEL_SYNS.items() if any(t in lvl_txt for t in v)),None)
    return lvl

# ─── public function: parse_degree(raw_str) ──────────────────────────
def parse_degree(raw:str):
    if not raw or not raw.strip():
        return {"level_code":None,"level_name":None,"major":None,"is_degree":False}
    txt=re.sub(r"[^a-z0-9: ]+"," ",raw.lower())
    lvl=_rule_level(txt) or _fuzzy_level(txt) or _embed_level(txt)
    # extract major phrase (if ":" present or " in <major>")
    major_raw=None
    if ":" in raw: major_raw=raw.split(":",1)[1].strip()
    else:
        m=re.search(r"\\bin\\s+(.+)",raw,re.I)
        if m: major_raw=m.group(1).strip()
    major=_canon_major(major_raw or "")
    if lvl is None: lvl=_mistral(raw)
    return {"level_code":lvl,
            "level_name":LEVEL_CANON.get(lvl),
            "major":major,
            "is_degree":bool(lvl is not None)}
"""
with open(parser_path,"w") as f: f.write(textwrap.dedent(parser_code))
print("✓ education_parser saved at utils/education_parser.py")


✓ education_parser saved at utils/education_parser.py


In [3]:
# Cell 2 – Spark session + bronze loader + register UDF
import os, re
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

spark = SparkSession.getActiveSession() or (
    SparkSession.builder.appName("EduETL_2021_08").config("spark.driver.memory","8g").getOrCreate()
)
spark.conf.set("spark.sql.debug.maxToStringFields",200)

spark.sparkContext.addPyFile(parser_path)
from utils.education_parser import parse_degree

# UDF schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType
edu_schema = StructType([
    StructField("level_code", IntegerType()),
    StructField("level_name", StringType()),
    StructField("major",      StringType()),
    StructField("is_degree",  BooleanType())
])
parse_degree_udf = udf(parse_degree, edu_schema)

# helper to find snapshot files
def find_pq(prefix):
    for fn in os.listdir("datamart/bronze"):
        if re.match(rf"^{prefix}_(2021[-_]0?8|0?8[-_]2021).*\\.parquet$", fn, re.I):
            return f"datamart/bronze/{fn}"
    raise FileNotFoundError(prefix)

resume_df = spark.read.parquet("datamart/bronze/resume_2021-8.parquet")
jd_df     = spark.read.parquet("datamart/bronze/jd_2021-8.parquet")
labels_df = spark.read.parquet("datamart/bronze/labels_2021-8.parquet")
print("Rows → resume:", resume_df.count(), "| jd:", jd_df.count(), "| labels:", labels_df.count())


25/06/15 07:08:21 WARN SparkContext: The path /app/utils/education_parser.py has been added already. Overwriting of added paths is not supported in the current version.


Rows → resume: 1038 | jd: 1038 | labels: 1038


In [4]:
# Cell 3 – build JD & resume edu tables (keep all raw cols + imputed level)
from pyspark.sql.functions import (explode_outer, col, when, lit, coalesce, upper)

# ─ JD table ────────────────────────────────────────────────────────
jd_edu = (
    jd_df
    .select(col("id").alias("job_id"), col("required_education").alias("required_education_raw"))
    .withColumn("edu", parse_degree_udf("required_education_raw"))
    .select("*",
            col("edu.level_code").alias("jd_level_raw"),
            col("edu.major").alias("jd_major_raw"))
    .withColumn(
        "jd_level_imp",
        when(col("jd_level_raw").isNotNull(), col("jd_level_raw"))
        .when(upper(col("jd_major_raw")).like("%MBA%"), lit(3))
        .when(upper(col("jd_major_raw")).like("%PHD%"), lit(4))
    )
)

# ─ Resume table ────────────────────────────────────────────────────
res_raw = (
    resume_df
    .select("id", explode_outer("education").alias("edu"))
    .selectExpr("id as resume_id",
                "edu.degree        as degree_raw",
                "edu.institution   as institution_raw",
                "edu.date_start    as date_start_raw",
                "edu.date_end      as date_end_raw",
                "edu.grade         as gpa_raw")
)

res_clean = (
    res_raw
    .withColumn("edu", parse_degree_udf("degree_raw"))
    .select("*",
            col("edu.level_code").alias("level_code_raw"),
            col("edu.major").alias("major_raw"))
)

res_ctx = (
    res_clean.alias("r")
    .join(labels_df.select("resume_id", "job_id", "fit_score"), "resume_id", "left")
    .join(jd_edu.select("job_id", "jd_level_imp"), "job_id", "left")
)

res_edu_imp = (
    res_ctx
    .withColumn(
        "level_code_imp",
        coalesce(
            "level_code_raw",
            when((col("fit_score")>=0.5) & col("jd_level_imp").isNotNull(),
                 col("jd_level_imp"))
        )
    )
    .withColumn(
        "level_code_imp",
        when(col("level_code_imp").isNull() & upper(col("major_raw")).like("%MBA%"), lit(3))
        .when(col("level_code_imp").isNull() & upper(col("major_raw")).like("%PHD%"), lit(4))
        .otherwise(col("level_code_imp"))
    )
)

print("Still-null level_code_imp rows:",
      res_edu_imp.filter(col("level_code_imp").isNull()).count())


[Stage 14:>                                                         (0 + 1) / 1]

Still-null level_code_imp rows: 562


In [8]:
# Cell 4 – null-percentage tables for education columns → CSV
import pandas as pd
from pyspark.sql.functions import trim, isnan, sum as _sum

def null_table(df, columns, label):
    present = [c for c in columns if c in df.columns]
    missing = sorted(set(columns) - set(present))
    if missing: print(f"⚠︎ {label}: skipping missing cols → {missing}")
    total = df.count()
    dtypes = dict(df.dtypes)
    exprs=[]
    for c in present:
        cond = col(c).isNull() | (trim(col(c))=="")
        if dtypes[c] in ("double","float"): cond = cond | isnan(col(c))
        exprs.append(_sum(when(cond,1).otherwise(0)).alias(c))
    counts = df.select(exprs).collect()[0].asDict()
    return pd.DataFrame({"dataset":label,"column":present,
                         "null_count":[counts[c] for c in present],
                         "total_rows":total,
                         "null_pct":[round(counts[c]/total*100,2) for c in present]})

resume_cols = ["level_code_raw","level_code_imp","major_raw",
               "institution_raw","date_start_raw","date_end_raw","gpa_raw"]
jd_cols     = ["jd_level_raw","jd_level_imp","jd_major_raw"]

resume_stats = null_table(res_edu_imp, resume_cols, "resume_edu")
jd_stats     = null_table(jd_edu,      jd_cols,    "jd_edu")

display(resume_stats)
display(jd_stats)

,dataset,column,null_count,total_rows,null_pct
0,resume_edu,level_code_raw,731,2094,34.91
1,resume_edu,level_code_imp,562,2094,26.84
2,resume_edu,major_raw,1745,2094,83.33
3,resume_edu,institution_raw,178,2094,8.50
4,resume_edu,date_start_raw,1597,2094,76.27
5,resume_edu,date_end_raw,891,2094,42.55
6,resume_edu,gpa_raw,1688,2094,80.61


,dataset,column,null_count,total_rows,null_pct
0,jd_edu,jd_level_raw,430,1038,41.43
1,jd_edu,jd_level_imp,430,1038,41.43
2,jd_edu,jd_major_raw,517,1038,49.81


In [6]:
print('heyo')

heyo


In [7]:
# # ╔══════════════════════════════════════════════════════════════════╗
# # ║  Cell – Save silver-level education tables to single-file CSVs   ║
# # ╚══════════════════════════════════════════════════════════════════╝

# import os, glob, shutil

# # -------------------------------------------------------------------
# # OPTION A – small/medium data → collect to Pandas (fast & simple)
# # -------------------------------------------------------------------
# # Uncomment if each DataFrame fits comfortably in driver memory.
# #
# # res_edu_imp.toPandas().to_csv("resume_edu_silver_2021-08.csv", index=False)
# # jd_edu     .toPandas().to_csv("jd_edu_silver_2021-08.csv",     index=False)
# # print("✓ CSVs written (Pandas collect).")
# #
# # -------------------------------------------------------------------
# # OPTION B – large data → distributed write, then re-name single part
# # -------------------------------------------------------------------

# def save_single_csv(df, out_name):
#     tmp_dir = out_name + "_tmp"
#     # coalesce(1) → exactly one part file
#     df.coalesce(1).write.csv(tmp_dir, header=True, mode="overwrite")
#     part_file = glob.glob(f"{tmp_dir}/part-*")[0]
#     shutil.move(part_file, out_name)
#     shutil.rmtree(tmp_dir)
#     print(f"✓ {out_name}  ({df.count():,} rows)")

# save_single_csv(res_edu_imp, "resume_edu_silver_2021-08.csv")
# save_single_csv(jd_edu,      "jd_edu_silver_2021-08.csv")
